## Исследование надёжности заёмщиков

### Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Структура исследования:
* Шаг 1. Загрузка данных и ознакомление с ними;
* Шаг 2. Предобработка данных;
- Обработка пропусков;
- Работа с некачественными данными;
- Замена типа данных;
- Обработка дубликатов;
- Лемматизация;
- Категоризация данных;
* Шаг 3. Ответы на вопросы исследования;
* Шаг 4. Общий вывод.

В ходе исследования применим следующие операции над данными:
* работа с пропусками;
* работа с дубликатами;
* изменение типа данных;
* лемматизация;
* классификация данных;
* построение сводных таблиц;
* применение конструкции try-except.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеку pandas для работы с табличными данными.

In [1]:
import pandas as pd

Прочитаем файл data.csv с данными о клиентах банка и сохраним его в переменной df.

In [3]:
df = pd.read_csv('/Users/polzovatel/Desktop/data.csv')

Выведем 5 случайных строк таблицы.

In [4]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11705,0,-8409.915019,41,среднее,1,в разводе,3,F,госслужащий,0,169140.738052,покупка жилья
10620,0,383541.132133,64,среднее,1,гражданский брак,1,F,пенсионер,0,135974.069965,сыграть свадьбу
11023,0,-1063.239925,39,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,141944.015162,получение высшего образования
4573,0,-107.957643,32,среднее,1,гражданский брак,1,M,сотрудник,0,102628.360734,на покупку подержанного автомобиля
11119,0,-1668.341542,41,среднее,1,женат / замужем,0,F,госслужащий,0,285524.865637,операции с жильем


Общая информация о данных таблицы df.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 21525 строк и 12 столбцов, типы данных у столбцов - float64, int64 и object.

Подробно разберём, какие в df столбцы и какую информацию они содержат:

* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

Можно заметить, что количество значений в столбцах различается. Это говорит о том, что в данных есть null значения. Причем количество пропусков одинаковое для столбцов days_employed и total_income. Возможно пропуски в этих столбцах встречаются в одних и тех строках. Проверим это в ходе работы над проектом.

Столбцы days_employed и total_income имеют тип данных float64. Стаж в днях не может быть дробным числом, а ежемесячный доход достаточно указать до целых значений (без копеек). Если нет острой необходимости работать с дробными числами, то разумнее и проще использовать целые. Нужно будет исправить тип данных этих столбцов на int64.

Взглянем на основные статистические параметры числовых данных в таблице с помощью метода describe().

In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Нелогичными являются отрицательные данные в столбце days_employed — стаж не может быть отрицательным. Кроме того, в строках встречается отрицательное значение количества детей в семье и возраст клиентов, равный нулю — такого тоже не может быть.

Взглянем на статистические параметры тесктовых данных в таблице с помощью метода describe() и аргумента include='O'.

In [6]:
df.describe(include='O')

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


Бросается в глаза, что имеется 15 уникальных значений уровней образования клиентов. Надо внимательно посмотреть, что это за значения. Также подозрительно выглядят 3 вида пола.

### Вывод

Каждая строка таблицы содержит информацию о клиенте банка. В данных были обнаружены такие проблемы:
* пропуски в столбцах days_employed и total_income;
* отрицательные значения в столбце days_employed и children;
* нулевые значения в столбце dob_years;
* большое количество уникальных значений в столбцах education и gender;
* вещественный тип данных в столбцах days_employed и total_income.

### Шаг 2. Предобработка данных

### Обработка пропусков

В ходе ознакомления с данными мы обнаружили, что в столбцах days_employed и total_income меньше значений, чем в других, то есть в них есть пропуски.

Проверим, какие есть вообще значения в этих столбцах. Подходящими нам являются числа, неподходящими - либо строки, либо пропуски. Посчитаем количество чисел, строк и пропусков с помощью конструкции try-except. Если значение можно привести к типу int, то оно является числом. Если нельзя, то для значения типа string появится ошибка IndentationError, а для пустого значения - ошибка ValueError.

Проведём расчёт и выведем по порядку количество числовых, строковых и пустых значений в столбце total_income.

In [7]:
num_count = 0
str_count = 0
nan_count = 0
for row in df['total_income']:
    try:
        num = int(row)
        num_count += 1
    except IndentationError:
        string = str(row)
        str_count += 1
    except ValueError: 
        nan_count += 1
num_count, str_count, nan_count

(19351, 0, 2174)

Мы выяснили, что в столбце total_income 19351 число, 0 строк и 2174 пустых значения.

Проведем такой же расчет для столбца days_employed.

In [8]:
num_count = 0
str_count = 0
nan_count = 0
for row in df['days_employed']:
    try:
        num = int(row)
        num_count += 1
    except IndentationError:
        string = str(row)
        str_count += 1
    except ValueError: 
        nan_count += 1
num_count, str_count, nan_count

(19351, 0, 2174)

В столбце days_employed аналогично 19351 число, 0 строк и 2174 пустых значения.

Можно предположить, что пропуски в столбцах total_income и days_employed встречаются в одних и тех же строках. Проверим эту гипотезу.

Выведем 5 случайных строк, где столбец days_employed пустой. Используем метод isnull() для поиска строк с пустыми значениями.

In [9]:
df[df['days_employed'].isnull()].sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
851,0,NaN,56,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,дополнительное образование
3551,0,NaN,53,среднее,1,в разводе,3,F,пенсионер,0,NaN,приобретение автомобиля
10765,0,NaN,46,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,заняться образованием
9395,0,NaN,28,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,заняться образованием
9491,0,NaN,46,среднее,1,в разводе,3,F,сотрудник,0,NaN,автомобили


Видно, что значения являются пустыми также и в столбце total_income. Возможно нам просто повезло найти такие строки с совпадениями пропусков. Чтобы узнать наверняка, в общих ли строках значения пустые в этих столбцах, сделаем следующие операции.

Выберем строки, где столбец days_employed пустой, а столбец total_income, наоборот, не пустой. И с помощью метода shape с индексом '0' посчитаем количество таких строк.

In [10]:
df[(df['days_employed'].isnull())&(df['total_income'].notnull())].shape[0]

0

Таких строк не оказалось. Теперь сделаем обратную проверку.
Выберем строки, где столбец total_income пустой, а столбец days_employed, наоборот, не пустой, и посчитаем количество таких строк.

In [11]:
df[(df['total_income'].isnull())&(df['days_employed'].notnull())].shape[0]

0

Таких строк также не оказалось. Это означает, что пропуски в этих столбцах встрчаются в одних и тех же строках.


Можно заметить, что в строках с пропусками в total_income и days_employed значения других столбцов самые разнообразные. Логики в пропущенных данных нет, а значит однозначно определить, чем они должны были быть заполнены, невозможно. Это логично, ведь трудовой стаж и ежемесячный доход - это индивидуальные показатели для каждого клиента. 

В таком случае мы можем либо удалить строки с пропущенными данными, либо заполнить их средними или медианными значениями. Удалить строки мы можем, если их количество относительно небольшое. Рассчитаем, сколько таких строк от общего числа строк в таблице.

In [12]:
df['days_employed'].isnull().sum() / df.shape[0]

0.10099883855981417

Получилось, что около 10% строк имеют пропуски. Это довольно большая часть от всего объёма данных. Если удалить её, результаты исследования могут оказаться менее достоверными. Поэтому попробуем восстановить пустые значения, например, заполнить их средними или медианными значениями.

Тут мы сталкиваемся с другой проблемой в данных в столбце days_employed - отрицательные значения. Изучим подробнее этот столбец и попробуем найти логику, где и при каких обстоятельствах встречаются такие значения.

Посчитаем, сколько всего отрицательных значений в столбце days_employed.

In [13]:
df[df['days_employed'] < 0]['days_employed'].count()

15906

Вспомним, что всего заполненных значений в этом столбце 19 351. То есть бóльшая часть данных занесена с ошибкой. Посмотрим внимательнее на отрицательные значения с точки зрения статистики.

In [14]:
df[df['days_employed'] < 0]['days_employed'].describe()

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

Отрицательные значения колеблются от - 18 388 до -24 дней.

Если оценивать значения по модулю, то они выглядят реалистично. Вероятно, что здесь просто ошибка со знаком и его можно отбросить, чтобы получить верные значения. 

Посмотрим внимательнее на положительные значения в этом столбце с точки зрения статистики.

In [15]:
df[df['days_employed'] > 0]['days_employed'].describe()

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

Положительные значения колеблются от 328 728 до 401 755 дней. Даже если работать каждый день, то 328 728 дней - это 900 лет трудового стажа. Таких долгожителей-трудяг история пока не знает.

Вероятно, в этих данных опечатка с точкой десятичного числа - она смещена на одну позицию вправо. Если вернут ее на место, то значения будут выглядеть реалистично.

Исправим данные в столбце: положительные значения уменьшим на порядок, а отрицательные значения приведём к абсолютным величинам.

Напишем функцию, которая будет положительные значения делить на 10 (то есть сместит точку в числах на одну цифру влево), а отрицательные значения переведёт в положительные. Назовём функцию days_employed_correction.

In [16]:
def days_employed_correction(days_employed):
    if days_employed > 0:
        return days_employed / 10
    return days_employed * (-1)

Теперь обновим столбец days_employed, применим к нему метод apply с аргументом функцией days_employed_correction.

In [17]:
df['days_employed'] = df['days_employed'].apply(days_employed_correction)

Проверим, что значения в столбце были скорректированы. Выведем информацию о значения в столбце с помощью метода describe.

In [18]:
df['days_employed'].describe()

count    19351.000000
mean      8432.176951
std      13258.771925
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      40175.540048
Name: days_employed, dtype: float64

Отлично! Теперь данные в столбце похожи на правду!

Вероятно, что в процессе выгрузки произошел технический сбой и данные были выгружены с ошибкой. Нужно обратить внимание инженеров на эту проблему.

<div class="alert alert-info" role="alert">
Вернёмся к пропускам в столбце days_employed. Изучим строки таблицы, где встречаются пропуски трудового стажа. Применим метод describe для оценки статистических параметров в столбцах с числовыми значениями.
<div>

In [19]:
df[df['days_employed'].isnull()].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


<div class="alert alert-info" role="alert">
Посмотрим на столбцы с текстовыми значениями. Применим метод describe с аргументом include='O'.
<div>

In [20]:
df[df['days_employed'].isnull()].describe(include='O')

,education,family_status,gender,income_type,purpose
count,2174,2174,2174,2174,2174
unique,12,5,2,5,38
top,среднее,женат / замужем,F,сотрудник,на проведение свадьбы
freq,1408,1237,1484,1105,92


<div class="alert alert-info" role="alert">
Можно заметить, что в строках, где встречаются пропуски в столбце days_employed, клиенты абсолютно разнородные: встречаются разные возрасты, типы занятости, уровни образования, семейные статусы, количество детей, цели кредита. Скорее всего пропуски появились случайным образом.

Единственное, что мы можем сделать в такой ситуации - пенсионерам указать медианный стаж среди пенсионеров. Медианный, а не средний, потому что эта статистическая мера центральной тенденции лучше отражает стаж - она защищает нас от искажений в результате выбросов в данных.
    
Для клиентов с остальными типами занятости указать медиану мы не можем - это было бы очень грубым усреднением. Кто-то только начинает свой профессиональный путь, а кто-то уже близок к пенсии. Поэтому для них оставим пока пропуски.
<div>

<div class="alert alert-info" role="alert">
Оценим изменчивость стажа среди пенсионеров.
    <div>

In [21]:
df[(df['income_type'] == 'пенсионер') & (df['days_employed'].notnull())]['days_employed'].describe()

count     3443.000000
mean     36500.349124
std       2106.960607
min      32872.872060
25%      34664.934615
50%      36521.330627
75%      38323.139687
max      40175.540048
Name: days_employed, dtype: float64

<div class="alert alert-info" role="alert">
Стаж среди пенсионеров колеблется от 32872 до 40175 дней. Разброс кажется не таким большим, поэтому мы можем рассчитать медианный средний стаж и заполнить им стажы у пенсионеров, где пропущены значения.
    <div>

<div class="alert alert-info" role="alert">
Посчитаем, у скольких пенсионеров не заполнен стаж.  
    <div>

In [22]:
df[(df['income_type'] == 'пенсионер') & (df['days_employed'].isnull())]['income_type'].count()

413

<div class="alert alert-info" role="alert">
Рассчитаем медианный стаж для пенсионеров.
<div>

In [23]:
median_days_employed = df[(df['income_type'] == 'пенсионер') & (df['days_employed'].notnull())]['days_employed'].median()
median_days_employed

36521.330626573115

<div class="alert alert-info" role="alert">
Заполним этим значением пропуски у пенсионеров. Для этого напишем функцию, которая будет находить строки, где в столбце days_employed пропуск, а в столбце income_type находится 'пенсионер'. Чтобы сравнить значение с NaN, импортируем метод isnan из библиотеки math.

Напишем функцию, назовём её days_employed_correction_2.
    <div>

In [24]:
from math import isnan
def days_employed_correction_2(row):
    income_type = row['income_type']
    days_employed = row['days_employed']
    if income_type == 'пенсионер' and isnan(days_employed): 
        return median_days_employed
    else:
        return days_employed

<div class="alert alert-info" role="alert">
Заполним медианным значением стажа пустые строки у пенсионеров.
    <div>

In [25]:
df['days_employed'] = df.apply(days_employed_correction_2, axis=1)

<div class="alert alert-info" role="alert">
Проверим, что функция сработала. Посчитаем пропуски в стаже у пенсионеров.
    <div>

In [26]:
df[(df['income_type'] == 'пенсионер') & (df['days_employed'].isnull())]['income_type'].count()

0

<div class="alert alert-info" role="alert">
Пропусков не оказалось, значит функция верно сработала!
    <div>

<div class="alert alert-info" role="alert">
Теперь вернёмся к пропускам в столбце total_income. Заполним их медианными значениями ежемесячного дохода исходя из типа занятости. Для этого сначала нужно посчитать медианы для каждого типа занятости. Выведем все типы занятости.
    <div>

In [27]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

<div class="alert alert-info" role="alert">
Рассчитаем медианный доход для каждого типа занятости. 
    <div>

<div class="alert alert-info" role="alert">
Медианный доход сотрудников:
    <div>

In [28]:
median_income_employee = df[df['income_type'] == 'сотрудник']['total_income'].median()
median_income_employee

142594.39684740017

<div class="alert alert-info" role="alert">
Медианный доход компаньонов:
    <div>

In [29]:
median_income_companion = df[df['income_type'] == 'компаньон']['total_income'].median()
median_income_companion

172357.95096577113

<div class="alert alert-info" role="alert">
Медианный доход пенсионеров:
    <div>

In [30]:
median_income_pensioner = df[df['income_type'] == 'пенсионер']['total_income'].median()
median_income_pensioner

118514.48641164352

<div class="alert alert-info" role="alert">
Медианный доход госслужащих:
    <div>

In [31]:
median_income_civil_servant = df[df['income_type'] == 'госслужащий']['total_income'].median()
median_income_civil_servant

150447.9352830068

<div class="alert alert-info" role="alert">
Медианный доход безработных:
    <div>

In [32]:
median_income_unemployed = df[df['income_type'] == 'безработный']['total_income'].median()
median_income_unemployed

131339.7516762103

<div class="alert alert-info" role="alert">
Медианный доход предпринимателей:
    <div>

In [33]:
median_income_entrepreneur = df[df['income_type'] == 'предприниматель']['total_income'].median()
median_income_entrepreneur

499163.1449470857

<div class="alert alert-info" role="alert">
Медианный доход студентов:
    <div>

In [34]:
median_income_student = df[df['income_type'] == 'студент']['total_income'].median()
median_income_student

98201.62531401133

<div class="alert alert-info" role="alert">
Медианный доход клиентов в декрете:
    <div>

In [35]:
median_income_decree = df[df['income_type'] == 'в декрете']['total_income'].median()
median_income_decree

53829.13072905995

<div class="alert alert-info" role="alert">
Получились довольно высокие медианные доходы даже по меркам Москвы. Можно предположить, что банк даёт кредит обеспеченным людям. Видимо, чтобы сократить процент невозвратов.
    <div>

<div class="alert alert-info" role="alert">
Теперь напишем функцию, которая будет искать пропуски в столбце total_income и, исходя из типа занятости в столбце income_type, возвращать подходящий медианный доход. Назовём функцию total_income_correction.
    <div>

In [36]:
def total_income_correction(row):
    income_type = row['income_type']
    total_income = row['total_income']
    if isnan(total_income):
        if income_type == 'сотрудник': 
            return median_income_employee
        elif income_type == 'компаньон':
            return median_income_companion
        elif income_type == 'пенсионер':
            return median_income_pensioner
        elif income_type == 'госслужащий':
            return median_income_civil_servant
        elif income_type == 'безработный':
            return median_income_unemployed
        elif income_type == 'предприниматель':
            return median_income_entrepreneur
        elif income_type == 'студент':
            return median_income_student
        elif income_type == 'в декрете':
            return median_income_decree
    else:
        return total_income   

<div class="alert alert-info" role="alert">
Проверим работу функции на нескольких примерах. Для этого создадим таблицу из заголовков и одной строки. Чтобы задать пустое значение, импортурем библиотеку numpy.
    <div>

In [37]:
import numpy as np
row_values = [np.nan, 'госслужащий'] 
row_columns = ['total_income', 'income_type']
row = pd.Series(data=row_values, index=row_columns)
total_income_correction(row)

150447.9352830068

In [38]:
row_values = [np.nan, 'студент'] 
row_columns = ['total_income', 'income_type']
row = pd.Series(data=row_values, index=row_columns)
total_income_correction(row)

98201.62531401133

<div class="alert alert-info" role="alert">
Функция работает верно. Доходы соответствуют рассчитанным выше значениям.
    <div>

<div class="alert alert-info" role="alert">
Заполним медианными значениеми доходов пустые строки в столбце total_income. Применим метод apply с аргументом функцией total_income_correction.
    <div>

In [39]:
df['total_income'] = df.apply(total_income_correction, axis=1)

<div class="alert alert-info" role="alert">
Проверим, что пустых значений в столбце total_income больше нет. 
    <div>

In [40]:
df['total_income'].isnull().sum()

0

<div class="alert alert-info" role="alert">
    Отлично! Клиентов без дохода не осталось!
<div>

### Вывод

<div class="alert alert-info" role="alert">
В ходе исследования данных были обнаружены пропуски в столбцах days_employed и total_income примерно в 10% строк таблицы. В столбце days_employed было принято решение заменить пропуски у пенсионеров на медианный стаж среди пенсионеров, у которых данные присутствуют. Для остальных клиентов мы оставили пропуски. В столбце total_income мы заменили пропуски на медианные ежемесячные доходы исходя из типа занятости клиентов.
    
Какая могла быть причина пропусков? Скорее всего произошла ошибка в выгрузке данных или при их записи в БД. Об этой проблеме нужно сообщить инженеру. Пропуски в данных могут быть случайными и нет. В нашем случае пропуски скорее всего являются случайными, а значит их причиной могут быть проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.
    <div>

### Работа с некачественными данными

В ходе первичного знакомства с данными мы обнаружили отрицательные значения в столбце children. Узнаем, сколько всего таких значений. Выведем все уникальные значения столбца методом value_counts().

In [41]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы обнаружили следующие артефакты: 
* отрицательное количество детей, а именно -1 ребенок у 47 клиентов;
* 20 детей у 76 клиентов. 20 детей, конечно, может быть у человека, но неправдоподобно выглядит разрыв между соседними значениями количества детей. У клиентов встречаются 1, 2, 3, 4, 5 детей, а дальше сразу 20. Причем такое количество встречается у 76 клиентов, это больше, чем клиентов с 4 и 5 детьми вместе взятых.

Можно предположить, что ошибки появились при некорректном вводе данных. Например, минус мог появится, потому что набирали дефис/тире. А ноль в значении '20' - случайно, как опечатка. Если бы мы были уверены в этом предположении, то могли бы восстановить данные в ячейках. Однако, так как мы не уверены, и так как строк с артефактами лишь небольшая часть от всей таблицы, то их можно безболезненно удалить.

Удалим строки со значениями детей '-1' и '20'. Для этого сначала найдём индексы строк, где встречаются такие значения. Применим метод index к строкам с ошибками.

In [42]:
wrong_children_indexes = df[(df['children'] == -1) | (df['children'] == 20)].index
wrong_children_indexes

Int64Index([  291,   606,   705,   720,   742,   800,   941,  1074,  1363,
             1929,
            ...
            20038, 20355, 20393, 20717, 21008, 21140, 21325, 21390, 21404,
            21491],
           dtype='int64', length=123)

Всего 123 строки с такими значениями. Теперь удалим эти строки методом drop(), аргументом для которого будет список индексов.

In [43]:
df = df.drop(wrong_children_indexes)

Чтобы окончательно удостовериться, что строки с количеством детей '-1' и '20' были удалены, выведем все уникальные значения столбца children методом value_counts().

In [44]:
df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Таких строк больше осталось. Проблема с детьми решена!

Теперь изучим значения в столбце education. При первоначальном знакомстве с данными нам показалось, что там излишне большое количество уникальных значений. 

Выведем уникальные значения этого столбца.

In [45]:
df['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что многообразие объясняется регистром, которым набирали значения. Это может усложнить работу с данными в будущем, поэтому приведём их в порядок. Применим метод str.lower() к столбцу education и приведём значения к нижнему регистру.

In [46]:
df['education'] = df['education'].str.lower()

Снова выведем уникальные значения этого столбца, чтобы проверить, что метод сработал.

In [47]:
df['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Так-то лучше! Теперь осталось только 5 уникальных значений уровня образования - все разные по смыслу.

Теперь изучим значения в столбце gender. При первоначальном знакомстве с данными мы там обнаружили три вида пола.

Выведем уникальные значения этого столбца.

In [48]:
df['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

У одного из клиентов указан пол 'XNA'. Не исключено, что эта аббревиатура что-то означает и была указана специально, однако более вероятно, что это просто ошибка ввода данных. Установить истинный пол мы не можем, так как неизвестно имя клиента. Поэтому удалим эту строку, тем более что она всего одна в таблице.

Найдем индекс этой строки. Применим метод index к строке с ошибкой.

In [49]:
wrong_gender_indexes = df[df['gender'] == 'XNA'].index
wrong_gender_indexes

Int64Index([10701], dtype='int64')

Теперь удалим эту строку методом drop(), аргументом для которого будет список индексов, в нашем случае состоящий из одного элемента.

In [50]:
df = df.drop(wrong_gender_indexes)

Чтобы окончательно удостовериться, что строка с полом 'XNA' удалена, выведем все уникальные значения столбца gender методом value_counts().

In [51]:
df['gender'].value_counts()

F    14154
M     7247
Name: gender, dtype: int64

Строка с ошибкой удалена. Осталось два пола - консерваторы торжествуют!

Теперь изучим значения в столбце dob_years. При первоначальном знакомстве с данными мы заметили, что есть клиенты с возрастом 0 лет. Выведем все уникальные значения возрастов до 18 лет и более 120 лет, возможно такие возраста тоже встречаются в таблице. Первым кредит не положен, а вторые, к сожалению, совсем малочисленны в обществе, чтобы попасть в нашу таблицу.

In [52]:
df[(df['dob_years'] < 18) | (df['dob_years'] > 120)]['dob_years'].value_counts()

0    100
Name: dob_years, dtype: int64

Клиентов, которым 0 лет, аж 100 человек. Других "неподходящих" возрастов нет. Ноль лет - это явно ошибка. К сожалению, точно восстановить эти данные мы не можем. Но в теории можно для клиентов с типом занятости 'пенсионер' выставить медианный возраст среди других пенсионеров в таблице, а для остальных клиентов - медианный возраст среди клиентов с другим типом занятости. Так как строк с ошибками немного, то просто удалим их из таблицы.

Найдем индексы строк с ошибками, применив метод index.

In [53]:
wrong_years_indexes = df[df['dob_years'] == 0].index
wrong_years_indexes

Int64Index([   99,   149,   270,   578,  1040,  1149,  1175,  1386,  1890,
             1898,  2082,  2284,  2469,  2487,  2870,  3044,  3147,  3704,
             4064,  4147,  4922,  4930,  5014,  6071,  6407,  6411,  6670,
             6778,  6831,  6848,  6859,  7034,  7073,  7075,  7252,  7344,
             8061,  8416,  8574,  8672,  8784,  9062,  9151,  9588, 10163,
            10188, 10306, 10364, 10384, 10545, 10595, 11077, 11289, 11468,
            11481, 11576, 11664, 11990, 12062, 12225, 12297, 12403, 12729,
            13117, 13187, 13232, 13413, 13423, 13439, 13521, 13741, 13968,
            14477, 14514, 14608, 14659, 15140, 15295, 15409, 15433, 15886,
            15891, 16042, 16861, 16922, 17215, 17306, 17613, 18176, 18415,
            18539, 18732, 18851, 19116, 19371, 19829, 20462, 20577, 21179,
            21313],
           dtype='int64')

Теперь удалим эти строки методом drop(), аргументом для которого будет список индексов.

In [54]:
df = df.drop(wrong_years_indexes)

Чтобы окончательно удостовериться, что клиенты с возрастом '0' были удалены, попробуем вывести строки с ошибками.

In [55]:
df[df['dob_years'] == 0]['dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

Строки с ошибками удалены! Таблица становится всё более годной для работы с ней: проверки гипотез и поисков инсайтов.

### Вывод

Были обнаружены некачественные данные в столбцах children, education, gender и dob_years. Ошибки в них больше похожи на некорректный ввод. Это повод настроить "защиту от дурака" в интерфейсах заполнения данных. Тогда в будщем такие проблемы в данных устранять не придётся.

### Замена типа данных

При знакомстве с данными мы обратили внимание, что данные в столбце с ежемесячным доходом имеют вещественный тип. 

Выведем несколько случайных данных из столбца total_income.

In [90]:
df['total_income'].sample(5)

7691     141067.943609
152      143113.329293
14537    107020.027874
12160    278191.983457
3638     137514.972360
Name: total_income, dtype: float64

Цифры после запятой в данном случае избыточны, потому что информация о копейках и долях копеек не принесёт нам дополнительной пользы. Если нет острой необходимости работать с дробными числами, то разумнее и проще использовать целые.

Переведём тип данных в столбце total_income в целочисленный, используя метод astype() с аргументом int. Этот метод позволяет перевести данные в нужный тип, просто указав его в качестве аргумента.

In [91]:
df['total_income'].astype('int')

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21225    224791
21226    155999
21227     89672
21228    244093
21229     82047
Name: total_income, Length: 21230, dtype: int64

Проверим, что тип данных изменился. Выведем несколько случайных данных из столбца total_income.

In [89]:
df['total_income'].sample(5)

8658     124293.934077
7521     172357.950966
2439     353543.857480
20124    181470.172676
11009    118514.486412
Name: total_income, dtype: float64

Видно, что числа стали целыми, тип данных изменился на int64. Отлично! Задача решена!

### Вывод

Неподходящий тип данных для значений ежемесячного дохода скорее всего изначально настроен в базе данных. Нужно понять, специально ли там выставлен такой тип данных, используется ли в каких-либо задачах дробная часть, или это просто ошибка в настройках соответствующего столбца в БД. Если разумнее для всех задач использовать целые числа дохода клиентов, то в базе данных нужно будет исправить тип данных. Тогда проблема в будущем не появится.

Иногда нам может казаться, что тип данных указан нелогично, потому что мы оцениваем их с точки зрения своих задач. Однако, зная изначально почему в БД указан именно такой тип данных, зная весь спектр задач, где применяются эти данные, нам станет ясен выбор инженеров. 

### Обработка дубликатов

Необходимо установить наличие дубликатов в таблице. Если найдутся, удалим их, и проверим, все ли удалились.

Получим суммарное количество дубликатов в таблице df. Применим к таблице метод duplicated() для выявления строк-дублей и метод sum(), чтобы найти суммарное количество дубликатов. Работают эти два метода следующим образом: метод duplicated() выводит таблицу с булевыми значениями True и False для каждой строки в зависимости от того, является ли строка дублем или нет. А метод sum() посчитает суммарное количество дубликатов в этой таблице.

In [59]:
df.duplicated().sum()

71

Удалим все дубликаты из таблицы df методом drop_duplicates(). При удалении индексы оставшихся строк не изменятся. Таким образом, порядок следования индексов строк будет с пропусками. Чтобы обновить индексы после удаления дублей применим метод reset_index(drop = True).

In [60]:
df = df.drop_duplicates().reset_index(drop = True)

Проверим таблицу на отсутствие дубликатов.

In [61]:
df.duplicated().sum()

0

Теперь таблица очищена от дублей!

### Вывод

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит разобраться с причинами появления такого «информационного мусора» и сообщить о проблеме инженерам.

### Лемматизация

При предварительном изучении данных мы могли увидеть, что в столбце с целями получения кредитов 38 уникальных значений. Имеет смысл проверить, действительно ли это всё разные по смыслу цели или где-то встречаются записанные по-разному одни и те же цели. Для этого посмотрим внимательно на уникальные значения в столбце purpose. Применим метод value_counts() к этому столбцу.

In [62]:
df['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           755
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      612
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

Как мы и предположили, в этом столбце встречаются одинаковые по смыслу, но по-разному записанные значения, например, 'свадьба', 'на проведение свадьбы', 'сыграть свадьбу'. Выделим леммы (то есть приведём слова к словарным формам), чтобы понять, какие уникальные слова встречаются в значениях столбца purpose и как часто.

Для этого импортируем библиотеку с функцией лемматизации на русском языке — pymystem3.

In [63]:
from pymystem3 import Mystem
m = Mystem() 

Сохраним все уникальные цели, на которые были выданы кредиты, в переменной text. Для этого используем метод unique() к столбцу purpose.

In [64]:
text = df['purpose'].unique()

Метод unique() вернёт уникальные значения в виде списка строк. Преобразуем этот список в одну строку методом ' '.join(), а в качестве разделителя элементов списка укажем пробел.

In [65]:
text = ' '.join(text)

Выделим леммы слов, применив метод lemmatize(). Этот метод возвращает список из лемм. Затем посчитаем количество упоминаний лемм среди причин для получения кредита. Для этого вызовем специальный контейнер Counter из модуля collections. Counter вернёт нам словарь, где в качестве ключей будут указаны леммы, а в качестве значений - количество упоминаний слов с такими леммами.

In [66]:
lemmas = m.lemmatize(text)
from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

### Вывод

Можно увидеть, что банк выдаёт кредит на операции с недвижимостью, покупку автомобиля, на обучение и свадьбу. Как мы и предположили, целей для кредита не 38, а гораздо меньше. Будет разумно единые по смыслу причины привести и к единому написанию. Для этого в интерфейсе ввода данных можно вместо строки ввода сделать, например, выпадающий список.

Каких-то небольших потребительских кредитов банк не выдаёт. Это подтверждает гипотезу, что банк работает только с обеспеченными клиентами - таким клиентам нужны кредиты только под крупные нужды. Из-за мелочей они за кредитом не пойдут.

### Категоризация данных

Оценим, как влияют разного рода категории клиентов на возврат кредита вовремя.

Сначала оценим, сколько всего клиентов нарушали срок возврата. Применим метод value_counts() к строке debt.

In [67]:
df['debt'].value_counts()

0    19506
1     1724
Name: debt, dtype: int64

Посчитаем процент просрочивших выплату клиентов.

In [68]:
df[df['debt'] == 1]['debt'].count() / df['debt'].count()

0.08120584079133301

Примерно 8% клиентов нарушили сроки. 

#### Категоризация клиентов по целям получения кредита

Мы выяснили, что существуют 4 категории причин, по которым банк выдаёт кредиты: операции с недвижимостью, покупка автомобиля, обучение и свадьба. Чтобы мы могли оценить, как влияет причина кредита на его возврат в срок, нам нужно провести разделение клиентов на категории по причинам. 

Для этого напишем функцию, которая на вход будет принимать значение в столбце purpose, то есть причину кредита. Затем функция будет находить леммы в этом значении и соотносить их с ключевым словом. По этому слову мы будем относить клиента к той или иной категории.

Так как мы выделили 4 категории причин, то для каждой категории определим ключевое слово:
* для категории 'операции с недвижимостью' - слово 'жилье' или 'недвижимость';
* для категории 'свадьба' - слово 'свадьба';
* для категории 'образование' - слово 'образование';
* для категории 'автомобиль' - слово 'автомобиль'.

Если ни одно из этих ключевых слов не будет найдено, то отнесём этого клиента временно к категории else. Затем проверим, что мы упустили, почему клиент не попал ни в одну из 4 категорий.

Итак, напишем функцию. Назовём её purpose_group.

In [69]:
def purpose_group(purpose):
    list_of_lemmas = m.lemmatize(purpose)
    if 'жилье' in list_of_lemmas or 'недвижимость' in list_of_lemmas:
        return 'операции с недвижимостью'
    if 'свадьба' in list_of_lemmas:
        return 'свадьба'
    if 'образование' in list_of_lemmas:
        return 'образование'
    if 'автомобиль' in list_of_lemmas:
        return 'автомобиль'
    return 'else'

Теперь создадим новый столбец в таблице, куда будут добавлены значения категорий. Для этого применим метод apply() к столбцу purpose, а в качестве аргумента метода выбираем функцию purpose_group.

Новый столбец назовём purpose_group. 

In [70]:
df['purpose_group'] = df['purpose'].apply(purpose_group) 

Выведем несколько случайных строк из таблицы, чтобы проверить, что появился новый столбец и он заполнен значениями.

In [71]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
20386,1,NaN,53,высшее,0,женат / замужем,0,F,компаньон,0,172357.950966,операции с недвижимостью,операции с недвижимостью
17530,0,2919.524476,32,среднее,1,гражданский брак,1,F,сотрудник,0,188768.908149,профильное образование,образование
7976,0,3865.452926,41,высшее,0,женат / замужем,0,F,госслужащий,0,55045.116058,на покупку автомобиля,автомобиль
6650,0,4918.645374,58,среднее,1,женат / замужем,0,M,сотрудник,0,189811.318493,жилье,операции с недвижимостью
20413,0,36170.098781,63,среднее,1,в разводе,3,F,пенсионер,0,399158.253071,на покупку автомобиля,автомобиль


Столбец purpose_group появился, значения в строках заполнены. Проверим, есть ли для какого-либо клиента значение else в этом столбце.

In [72]:
df['purpose_group'].value_counts()

операции с недвижимостью    10703
автомобиль                   4258
образование                  3970
свадьба                      2299
Name: purpose_group, dtype: int64

Значений else нет. Это означает, что мы правильно провели лемматизацию значений в столбце purpose и определили основные категории причин, по которым банк выдаёт кредит.

#### Категоризация клиентов по уровню ежемесячного дохода

Разделим клиентов на категории исходя из их уровня доходов. Найдём квартили, которые делят всех клиентов на 4 равные по количеству клиентов категории. Для этого применим метод describe() к столбцу total_income. Чтобы значения квартилей были целыми, применим метод astype с аргументом int.

In [73]:
df['total_income'].describe().astype('int')

count      21230
mean      165384
std        98425
min        20667
25%       107550
50%       142594
75%       195813
max      2265604
Name: total_income, dtype: int64

Теперь мы знаем границы доходов клиентов, по которым можем их разделить на 4 равные категории. Проведем разделение на категории.

Для этого напишем функцию, которая на вход будет принимать значение в столбце total_income. Затем сравнивать его с квартилями и относить к одной из категорий. Категории получатся такие:
* 'категория 1' - 25% клиентов, у которых самый низкий уровень дохода;
* 'категория 2' - 25% клиентов, у которых уровень дохода ниже среднего, но выше, чем в категории 1;
* 'категория 3' - 25% клиентов, у которых уровень дохода выше среднего, но ниже, чем в категории 4;
* 'категория 4' - 25% клиентов, у которых самый высокий уровень дохода.

Итак, напишем функцию. Назовём её income_group.

In [74]:
def income_group(total_income):
    if total_income < 107550:
        return 'категория 1'
    if total_income < 145017:
        return 'категория 2'
    if total_income < 195795:
        return 'категория 3'
    return 'категория 4'

Проверим, что функция работает и адекватно возвращает категорию.

In [75]:
income_group(107550)

'категория 2'

In [76]:
income_group(201250)

'категория 4'

Функция работает верно.

Теперь создадим новый столбец в таблице, куда будут добавлены значения категорий. Для этого применим метод apply() к столбцу total_income, а в качестве аргумента метода выбираем функцию income_group.

Новый столбец назовём income_group.

In [77]:
df['income_group'] = df['total_income'].apply(income_group) 

Выведем несколько случайных строк из таблицы, чтобы проверить, что появился новый столбец и он заполнен значениями.

In [78]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_group
19609,0,1323.624855,29,высшее,0,Не женат / не замужем,4,M,сотрудник,0,166532.255501,заняться высшим образованием,образование,категория 3
17517,0,493.666263,20,среднее,1,Не женат / не замужем,4,M,компаньон,0,226947.057213,ремонт жилью,операции с недвижимостью,категория 4
11056,1,1980.791140,49,высшее,0,гражданский брак,1,F,сотрудник,0,201132.701767,свадьба,свадьба,категория 4
10389,2,2636.753966,46,среднее,1,женат / замужем,0,M,сотрудник,0,125600.372094,покупка коммерческой недвижимости,операции с недвижимостью,категория 2
5762,1,467.152556,29,среднее,1,гражданский брак,1,M,сотрудник,0,64194.457705,покупка жилья для семьи,операции с недвижимостью,категория 1


Столбец income_group появился, значения в строках заполнены.

### Шаг 3. Ответьте на вопросы

- Как разные цели кредита влияют на его возврат в срок?

Мы уже провели категоризацию клиентов исходя из их целей получения кредита. Теперь выясним, как категория влияет на возврат кредита в срок.

<div class="alert alert-info" role="alert">
Сгруппируем таблицу по столбцу purpose_group, применив метод groupby. Затем применим метод agg, а в качестве его аргумента зададим словарь, где ключём будет столбец debt, а значениями - функции группировки данных в столбце debt - расчёт среднего (mean), суммы значений(sum) и количества значений (count).
    <div>

In [79]:
df.groupby('purpose_group').agg({'debt': ['mean', 'sum', 'count']})

debt            
                              mean  sum  count
purpose_group                                 
автомобиль                0.093236  397   4258
образование               0.092947  369   3970
операции с недвижимостью  0.072596  777  10703
свадьба                   0.078730  181   2299

<div class="alert alert-info" role="alert">
Оценим объёмы получившихся групп. По каждой цели кредита получилось достаточно большое количество клиентов - столбец count. В столбце sum мы можем видеть, сколько из них нарушили сроки возврата. А в столбце mean рассчитаное среднее значение количества нарушений.
    
Для того, чтобы оценить различия в группах с точки зрения статистической значимости, проводять анализ различий между группами. Сейчас мы этого делать не будем. 
    <div>

### Вывод

Можно заметить, что те клиенты, которые берут кредит на покупку автомобиля и образование, нарушают сроки возврата кредита чаще, чем те клиенты, которые берут кредит на операции с недвижимостью и свадьбу. Это повод пересмотреть условия выдачи кредитов на покупку автомобиля и образование, чтобы уменьшить количество нарушений сроков возврата.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Мы уже провели категоризацию клиентов исходя из их уровня дохода. Теперь выясним, как категория влияет на возврат кредита в срок.

Сгруппируем таблицу, состоящую из столбцов income_group и debt, по значениям в столбце income_group. Для этого применим метод groupby к столбцу income_group. При группировке мы хотим рассчитать средние значения в столбце debt. Для этого применим метод mean().

In [80]:
df[['income_group', 'debt']].groupby('income_group').mean()

,debt
income_group,
категория 1,0.080068
категория 2,0.088313
категория 3,0.084590
категория 4,0.071577


### Вывод

Реже всех нарушают сроки возврата кредита самые обеспеченные клиенты. Следующие за ними - менее всего обеспеченные клиенты. Клиенты, которые получают близкие к среднему ежемесячному доходу, нарушают чаще всего. 

Исходя из этих данных, можно выдвинуть предположение, что для самых обеспеченных клиентов возврат кредита не является проблемой. А менее всего обеспеченные клиенты ответственно относятся к возврату кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим, какие в нашей таблице встречаются семейные положения и как часто. Выведем все уникальные значения в столбце family_status. Применим метод value_counts().

In [81]:
df['family_status'].value_counts()

женат / замужем          12213
гражданский брак          4112
Не женат / не замужем     2780
в разводе                 1179
вдовец / вдова             946
Name: family_status, dtype: int64

<div class="alert alert-info" role="alert">
Сгруппируем таблицу по столбцу family_status, применив метод groupby. Затем с помощью метода agg рассчитаем средние значения (mean), суммы значений (sum) и количества значений (count) в столбце debt.
    <div>

In [82]:
df.groupby('family_status').agg({'debt': ['mean', 'sum', 'count']})

debt            
                           mean  sum  count
family_status                              
Не женат / не замужем  0.097842  272   2780
в разводе              0.071247   84   1179
вдовец / вдова         0.065539   62    946
гражданский брак       0.093142  383   4112
женат / замужем        0.075575  923  12213

<div class="alert alert-info" role="alert">
Оценим объёмы получившихся групп. По каждой категории семейного положения получилось достаточно большое количество клиентов - столбец count. В столбце sum мы можем видеть, сколько из них нарушили сроки возврата. А в столбце mean рассчитаное среднее значений количества нарушений.
    <div>

### Вывод

Чаще всего нарушают сроки возврата клиенты, которые не имеют и никогда не имели формализованных супружеских отношений. А клиенты, которые прошли через брак или сейчас в браке, более ответственно относятся к возврату кредита в срок. Вдовы и вдовцы реже всех нарушают сроки.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Еще раз вспомним, сколько детей у наших клиентов. Выведем все уникальные значения в столбце children. Применим метод value_counts().

In [83]:
df['children'].value_counts()

0    14021
1     4792
2     2039
3      328
4       41
5        9
Name: children, dtype: int64

<div class="alert alert-info" role="alert">
Сгруппируем таблицу по столбцу children, применив метод groupby. Затем с помощью метода agg рассчитаем средние значения (mean), суммы значений (sum) и количества значений (count) в столбце debt.
    <div>

In [84]:
df.groupby('children').agg({'debt': ['mean', 'sum', 'count']})

debt             
              mean   sum  count
children                       
0         0.075458  1058  14021
1         0.092028   441   4792
2         0.095145   194   2039
3         0.082317    27    328
4         0.097561     4     41
5         0.000000     0      9

<div class="alert alert-info" role="alert">
Оценим объёмы получившихся групп. Категории с 4 и 5 детьми оказались небольшими - столбец  count. Чем меньше детей, тем больше по размеру становится группа. И тем ближе становятся средние значения возвратов кредита к своим истинным значениям (значениям в генеральной совокупности).
    <div>

### Вывод

<div class="alert alert-info" role="alert">
Реже всех нарушают сроки возврата клиенты без детей. Когда у клиентов появляются дети, они начинают чаще нарушать. Причем клиенты с 3 детьми оказались более ответственны, чем клиенты с 1 и 2 детьми. 

По поводу клиентов с 4 и 5 детьми, к сожалению, мы не можем делать выводов, так как выборки оказались достаточно малые и наши рассчитанные средние значения могут быть далеки от средних значений в генеральных совокупностях для этих групп.
<div>

- Как влияет семейные статус и наличие детей одновременно на возврат кредита в срок?

Чтобы ответить на этот вопрос, нужно построить сводную таблицу. Сгруппируем таблицу по семейному статусу и количеству детей и рассчитаем средние значение и количество возвратов кредита в срок.

Воспользуемся методом pivot_table().В качестве аргументов укажем:
- index — столбец, по которому группируем данные (family_status);
- columns — столбец, по значениям которого происходит группировка (children);
- values — значения, по которым мы хотим увидеть сводную таблицу (debt);
- aggfunc — функция, применяемая к значениям (mean и count).

In [85]:
df.pivot_table(index='family_status', columns='children', values='debt', aggfunc=['mean', 'count'])

mean                                               \
children                      0         1         2         3         4    5   
family_status                                                                  
Не женат / не замужем  0.093375  0.114094  0.121622  0.125000  0.500000  NaN   
в разводе              0.070785  0.067524  0.088608  0.090909  0.000000  NaN   
вдовец / вдова         0.061758  0.090909  0.150000  0.000000  0.000000  NaN   
гражданский брак       0.083579  0.119312  0.087977  0.142857  0.000000  0.0   
женат / замужем        0.069114  0.082210  0.095082  0.068826  0.103448  0.0   

                        count                                    
children                    0       1       2      3     4    5  
family_status                                                    
Не женат / не замужем  2249.0   447.0    74.0    8.0   2.0  NaN  
в разводе               777.0   311.0    79.0   11.0   1.0  NaN  
вдовец / вдова          842.0    77.0    20.0    6.0   1.0  NaN  
гражданский брак       2716.0   989.0   341.0   56.0   8.0  2.0  
женат / замужем        7437.0  2968.0  1525.0  247.0  29.0  7.0

Нулевые средние значения получились в тех группах, где было мало клиентов и среди них пока нет сорвавших срок возврата. Значения NaN означают, что нет таких групп, например, нет разведенных клиентов с 5 детьми. 5 детей есть только у клиентов в браке - официальном или гражданском. Видимо, 5 детей - это везкая причина сохранять отношения с партнёром. 

Качественно оценить средние значения можно только в тех группах, куда попало большое количество значений. Рассмотрим группы, где больше 100 клиентов. Для этого смотрим на правую часть сводной таблицы, на значения count.

Самыми ответственными оказались вдовы/вдовцы без детей, близко к ним - клиенты в разводе с 1 ребенком. Самые безответственные - женатые клиенты с 2 детьми и свободные клиенты - без партнеров и детей. 

### Шаг 4. Общий вывод

<div class="alert alert-info" role="alert">

В полученных данных были обнаружены пропуски, дубликаты и некачественные данные. Об этих проблемах нужно сообщить инженерам, чтобы они могли исправить ошибки. Тогда в будущем на предподготовку данных будет уходить меньше времени, а также можно будет делать более точные выводы исходя из данных. Ошибки в данных могут напрямую влиять на результат работы аналитика и его пользу для компании.

В ходе исследования мы выяснили, что нарушения возврата кредитов чаще встречаются среди клиентов с детьми (9,5%), без опыта формализованых супружеских отношений (9,8%), со средним уровнем дохода (8,8%), и которые берут кредит на покупку автомобиля или получение образования (9,3%). 

Нарушения встречаются реже среди клиентов без детей (7,5%), вдов/вдовцов (6,5%), с высоким уровнем дохода (7,1%) и которые берут кредит операции с недвижимостью (7,3%).

Эти выводы следует проверить на статистическую значимость. И если они окажутся значимыми, то сообщить о них менеджменту компании. Их можно будет использовать, чтобы оптимизировать систему скоринга клиентов и улучшить принятие решений о выдаче или невыдаче кредита.
    <div>